In [ ]:
from mxnet import nd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ts = 2
cr = 1.1
border = 3
density = 1000
ndays = 4
grid = 16 # 8 squares

fs = (grid-1)*ts + 2*border
nmice = round(fs*fs*density)
nmice

In [ ]:
nmice = 8

In [ ]:
traps = nd.stack(nd.repeat(nd.arange(start=-7.5, stop=8.5), repeats=grid),
                nd.tile(nd.arange(start=-7.5, stop=8.5), reps=grid))
traps = nd.transpose(traps)
traps = traps*ts # traps = nd.multiply(traps, ts) needed for context?

In [ ]:
mhomes = nd.random.uniform(low=-fs/2, high=fs/2, shape=(nmice,2)) # mice x xy
# mhomes

In [ ]:
forages = nd.random.normal(loc=0, scale=1, shape=(ndays,nmice,2)) # day x mice x xy
forages.shape
# forages

In [ ]:
forages = forages + mhomes
# forages

In [ ]:
# mouse 1 
forages[:,0,:]

In [ ]:
traps = traps.reshape(grid**2, 1, 1, 2)
traps.shape
#traps

In [ ]:
dist = nd.sqrt(nd.sum((traps - forages)**2, axis=3)) # trap x day x mouse
# dist

In [ ]:
dist

In [ ]:
import torch
import matplotlib.pyplot as plt

torch.manual_seed(0)

In [ ]:
ts = 2
cr = 1.1
border = 3
density = 1000
ndays = 4
grid = 16 # 8 squares

fs = (grid-1)*ts + 2*border
nmice = round(fs*fs*density)
nmice

In [ ]:
traps_torch = torch.stack((torch.linspace(-7.5, 7.5, grid).repeat_interleave(grid), 
                           torch.linspace(-7.5, 7.5, grid).repeat([grid])))
traps_torch = traps_torch.t()
traps_torch = traps_torch*ts
# traps_torch

In [ ]:
mhomes_torch = torch.empty(nmice,2)
mhomes_torch.uniform_(-fs/2, fs/2) # inplace rng
print(mhomes_torch.shape)
forages_torch = torch.normal(mean=torch.zeros(ndays,nmice,2), std=1) 
print(forages_torch.shape)
forages_torch = forages_torch + mhomes_torch
print(forages_torch.shape)

In [ ]:
traps_torch = traps_torch.reshape(grid**2, 1, 1, 2)
traps_torch.shape

In [ ]:
print((traps_torch - forages_torch).shape)
dist_torch = (traps_torch - forages_torch).pow(2).sum(dim=3).sqrt()
print(dist_torch.shape)

In [ ]:
values, indices = dist_torch.min(dim=0)
# print(values)
# print(indices)

In [ ]:
caught_mice = (values<cr).any(dim=0)
# caught_mice

In [ ]:
# Torch's argmax returns the last instance of the max value, not the first (like numpy). 
# Fortunately our simulations/distances are symmetric in time, so we can we can just look 
# reverse the day order and it SHOULD be the same. 
caught_days = (values[:,caught_mice]<cr).argmax(dim=0)
# caught_days

# indices[caught_days,caught_mice]

In [ ]:
# The (3 - day) is to reverse time to keep it consistent with our intuition 
idxs = (3 - caught_days)*grid*grid + indices[caught_days,caught_mice]
idxs

In [ ]:
traplayers_torch = torch.zeros(ndays,grid*grid)
traplayers_torch.put_(idxs, torch.tensor([1.]).expand_as(idxs), accumulate=True)
traplayers_torch = traplayers_torch.reshape(ndays, grid, grid)

In [ ]:
fig, axes = plt.subplots(2, 2) # row first
axes[0,0].imshow(traplayers_torch[0,:,:]) 
axes[0,1].imshow(traplayers_torch[1,:,:])
axes[1,0].imshow(traplayers_torch[2,:,:])
axes[1,1].imshow(traplayers_torch[3,:,:])

plt.show()

In [ ]:
plt.imshow(traplayers_torch.sum(dim=0))

Many image simulations

Start by generating the distance matrix with mxnet (for the gpu) then use pytorch to reduce it to the final image.

In [ ]:
from mxnet import nd
import mxnet as mx
import torch
import matplotlib.pyplot as plt

torch.manual_seed(0)
mx.random.seed(0)

torch.set_num_threads(11)

In [ ]:
numsim = 100000

trapspacing = torch.FloatTensor(numsim,).uniform_(0.1,5)
catchradius = torch.FloatTensor(numsim,).uniform_(0.1,5)
density = torch.FloatTensor(numsim,).uniform_(0.25,10)
border = 3
numdays = 4
grid = 16 # 8 squares

fieldsize = (grid-1)*trapspacing + 2*border
nummice = (fieldsize*fieldsize*density).round()
# nmice

In [ ]:
def gendistmat_torch(nmice, ts, fs, ndays=4, grid=16):
    traps_torch = torch.stack((torch.linspace(-7.5, 7.5, grid).repeat_interleave(grid), 
                               torch.linspace(-7.5, 7.5, grid).repeat([grid])))
    traps_torch = traps_torch.t()
    traps_torch = traps_torch*ts
    traps_torch = traps_torch.reshape(grid**2, 1, 1, 2)

    mhomes_torch = torch.empty(nmice,2)
    mhomes_torch.uniform_(-fs/2, fs/2) # inplace rng
    forages_torch = torch.normal(mean=torch.zeros(ndays,nmice,2), std=1) 
    forages_torch = forages_torch + mhomes_torch

    dist_torch = (traps_torch - forages_torch).pow(2).sum(dim=3).sqrt()

    return dist_torch

In [ ]:
# Seems to have an overflow issue if nmice = 1000000 fine if nmice = 100000
def gendistmat_nd(nmice, ts, fs, ctx, ndays=4, grid=16):
    # start by generating the traps
    traps = nd.stack(nd.repeat(nd.arange(start=-7.5, stop=8.5, ctx=ctx), repeats=grid),
                     nd.tile(nd.arange(start=-7.5, stop=8.5, ctx=ctx), reps=grid))
    traps = nd.transpose(traps)
    traps = traps*ts # traps = nd.multiply(traps, ts) needed for context?
    traps = traps.reshape(grid**2, 1, 1, 2)
    
    # generate the distance matrix
    mhomes = nd.random.uniform(low=-fs/2, high=fs/2, shape=(nmice,2), ctx=ctx) # mice x xy
    forages = nd.random.normal(loc=0, scale=1, shape=(ndays,nmice,2), ctx=ctx) # day x mice x xy
    forages = forages + mhomes
    dist = nd.sqrt(nd.sum((traps - forages)**2, axis=3)) # trap x day x mouse
    
    return dist.asnumpy()

In [ ]:
%%time
simimages = torch.empty(numsim,numdays,grid,grid)
for sim in torch.arange(numsim):
    # generate distance matrix
    dist_torch = gendistmat_torch(nmice=int(nummice[sim]), ts=float(trapspacing[sim]), fs=float(fieldsize[sim]))
    
    # reduce to image
    values, indices = dist_torch.min(dim=0)
    caught_mice = (values<catchradius[sim]).any(dim=0)
    # Torch's argmax returns the last instance of the max value, not the first (like numpy). 
    # Fortunately our simulations/distances are symmetric in time, so we can we can just look 
    # reverse the day order and it SHOULD be the same. 
    caught_days = (values[:,caught_mice]<catchradius[sim]).argmax(dim=0)
    # The (3 - day) is to reverse time to keep it consistent with our intuition 
    idxs = (3 - caught_days)*grid*grid + indices[caught_days,caught_mice]
    
    # make image
    traplayers_torch = torch.zeros(numdays,grid*grid)
    traplayers_torch.put_(idxs, torch.tensor([1.]).expand_as(idxs), accumulate=True)
    simimages[sim,:,:,:] = traplayers_torch.reshape(numdays, grid, grid)